In [1]:
import cv2
import numpy as np
import os
import sys

In [18]:
file_name = './fig/fish.jpg'

img = cv2.imread(file_name)

if img is None:
    print('img read err')
    sys.exit()

#caffe model
model = 'bvlc_googlenet.caffemodel'
config = 'deploy.prototxt'

net = cv2.dnn.readNet(model,config)

classNames = []
with open('classification_classes_ILSVRC2012.txt','rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    
### DL
# blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
blob = cv2.dnn.blobFromImage(img,1,(224,224),(104,117,123),
                            swapRB=False)

net.setInput(blob)

prob = net.forward()

# print(prob[:,:10])

out = prob.flatten()
classid =np.argmax(out)

confidence = out[classid]
print(confidence)
class_text = classNames[classid]
print(class_text)

text = f'{class_text} ({confidence*100:.2f}%)'
print(text)
    
cv2.putText(img,text,(10,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2,cv2.LINE_AA)

cv2.imshow('image',img)

cv2.waitKey()
cv2.destroyAllWindows()

0.98700094
anemone fish
anemone fish (98.70%)


In [3]:
img_list = os.listdir('./fig/')

file_name = []
for i in img_list:
    img_path = './fig/'+i
    file_name.append(img_path)

model = 'bvlc_googlenet.caffemodel'
config = 'deploy.prototxt'

net = cv2.dnn.readNet(model,config)

classNames = []
with open('classification_classes_ILSVRC2012.txt','rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

    
idx=0    
while True:
    img = cv2.imread(file_name[idx])
    
    if img is None:
        print('img read err')
        sys.exit()
    img=cv2.resize(img,(500,500))
    blob = cv2.dnn.blobFromImage(img,1,(224,224),(104,117,123),
                                swapRB=False)

    net.setInput(blob)

    prob = net.forward()

    out = prob.flatten()
    classid =np.argmax(out)
    confidence = out[classid]
    class_text = classNames[classid]
    text = f'{class_text} ({confidence*100:.2f}%)'
    cv2.putText(img,text,(10,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2,cv2.LINE_AA)

    cv2.imshow('image',img)

    if cv2.waitKey(3000)==27:
        break
    idx+=1
    if idx>=len(file_name):
        idx=0
cv2.destroyAllWindows()

In [43]:
img = cv2.imread('../face_detector/fig/cat123.jpg')

model = '../face_detector/opencv_face_detector_uint8.pb'
config = '../face_detector/opencv_face_detector.pbtxt.txt'
face_net = cv2.dnn.readNet(model,config)

blob = cv2.dnn.blobFromImage(img,1,(300,300),(104,117,123),swapRB=False)
face_net.setInput(blob)
out = face_net.forward()

detect = out[0,0,:,:]
h,w=img.shape[:2]
for i in range(detect.shape[0]):
    confidence = detect[i,2]
    if confidence>0.5:
        x1=int(detect[i,3]*w)
        y1=int(detect[i,4]*h)
        x2=int(detect[i,5]*w)
        y2=int(detect[i,6]*h)
        
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),2)
        text = 'face: {}%'.format(round(confidence*100,2))
        cv2.putText(img,text,(x1,y1-2),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1,cv2.LINE_AA)

cv2.imshow('image',img)

cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
model = '../face_detector/opencv_face_detector_uint8.pb'
config = '../face_detector/opencv_face_detector.pbtxt.txt'
face_net = cv2.dnn.readNet(model,config)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('open err')
    sys.exit()

while True:
    ret,frame = cap.read()
    
    if not ret:
        print('frame err')
        break
    
    frame = cv2.flip(frame,1)
    
    blob = cv2.dnn.blobFromImage(frame,1,(300,300),(104,117,123),swapRB=False)
    face_net.setInput(blob)
    out = face_net.forward()

    detect = out[0,0,:,:]
    h,w=frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i,2]
        if confidence>0.3:
            x1=int(detect[i,3]*w)
            y1=int(detect[i,4]*h)
            x2=int(detect[i,5]*w)
            y2=int(detect[i,6]*h)

            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
            text = 'face: {}%'.format(round(confidence*100,2))
            cv2.putText(frame,text,(x1,y1-2),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1,cv2.LINE_AA)

    cv2.imshow('frame',frame)
    
    if cv2.waitKey(20)==27:
        break
    
cap.release()
cv2.destroyAllWindows()